# Laboratorio 4

In [1]:
import pandas as pd
from pathlib import Path

Utilizarás el conjunto de datos de pokemon.

In [10]:
pkm = (
    pd.read_csv(Path().resolve().parent / "data" / "pokemon.csv", index_col="#")
    .rename(columns=lambda x: x.replace(" ", "").replace(".", "_").lower())
)
pkm.head()

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


## Ejercicio 1

(1 pto)

Agrupar por `generation` y `legendary` y obtener por grupo:

* Promedio de `hp`
* Mínimo y máximo de `sp_atk` y `sp_def`

In [3]:
pkm.groupby(["generation","legendary"]).agg(
    mean_hp=("hp","mean"),
    min_sp_atk=("sp_atk","min"),
    max_sp_atk=("sp_atk","max"),
    min_sp_def=("sp_def","min"),
    max_sp_def=("sp_def","max")
    )

mean_hp  min_sp_atk  max_sp_atk  min_sp_def  \
generation legendary                                                   
1          False       64.612500          15         175          20   
           True        98.000000          95         194          85   
2          False       69.613861          10         165          20   
           True       103.400000          90         115          75   
3          False       64.626761          10         165          20   
           True        81.666667          50         180          20   
4          False       69.740741          10         140          30   
           True       100.846154          75         150          70   
5          False       69.353333          15         145          25   
           True        96.133333          72         170          72   
6          False       66.189189          27         150          25   
           True        87.500000          81         170          90   

                      max_sp_def  
generation legendary              
1          False             130  
           True              125  
2          False             230  
           True              154  
3          False             135  
           True              200  
4          False             150  
           True              130  
5          False             135  
           True              129  
6          False             154  
           True              150

## Ejercicio 2

(1 pto)

El profesor Oakgueda determinó que una buen indicador de pokemones es: 

$$ 0.2 \, \textrm{hp} + 0.4 \,(\textrm{attack} + \textrm{sp_atk})^2 + 0.3 \,( \textrm{defense} + \textrm{sp_deff})^{1.5} + 0.1 \, \textrm{speed}$$

Según este indicador, ¿Qué grupo de pokemones (`type1`, `type2`) es en promedio mejor que el resto? 

In [4]:
def oakgueda_indicator(df):
    #display(df)
    temp_serie=df.apply(lambda row: 0.2*row["hp"]+0.4*(row["attack"]+row["sp_atk"])**2+0.3*(row["defense"]+row["sp_def"])**(1.5)+0.1*row["speed"],axis=1)
    #display(temp_serie)
    return temp_serie.mean()

In [16]:
type1,type2=pkm.groupby(["type1","type2"]).apply(oakgueda_indicator).idxmax()
print(f"En promedio, los pokemones {type1}-{type2} son mejores según el indicador del profesor Oakgueda")

En promedio, los pokemones Ground-Fire son mejores según el indicador del profesor Oakgueda


__Respuesta__: En promedio, los pokemones Ground-Fire son mejores según el indicador del profesor Oakgueda.

## Ejercicio 3

(1 pto)

Define una función que escale los datos tal que, si $s$ es una columna: 

$$s\_scaled = \frac{s - \min(s)}{\max(s) - \min(s)}$$

Y luego transforma cada columna numérica agrupando por si el pokemon es legendario o no.

In [22]:
def minmax_scale(s):
    return s-s.min()/s.max()-s.min()

pkm.groupby("legendary")[["hp","attack","defense","sp_atk","sp_def","speed"]].transform(lambda s: minmax_scale(s))

,hp,attack,defense,sp_atk,sp_def,speed
#,,,,,,
1,43.996078,43.972973,43.978261,54.942857,44.913043,39.968750
2,58.996078,56.972973,57.978261,69.942857,59.913043,54.968750
3,78.996078,76.972973,77.978261,89.942857,79.913043,74.968750
4,78.996078,94.972973,117.978261,111.942857,99.913043,74.968750
5,37.996078,46.972973,37.978261,49.942857,29.913043,59.968750
...,...,...,...,...,...,...
796,-0.333333,49.736842,129.900000,49.742268,129.900000,-0.277778
797,-0.333333,109.736842,89.900000,109.742268,89.900000,59.722222
798,29.666667,59.736842,39.900000,99.742268,109.900000,19.722222


## Ejercicio 4

(1 pto)

El profesor Oakgueda necesita saber cuántos pokemones hay luego de filtrar el dataset tal que el grupo de (`type1`, `type2`) tenga en promedio un indicador (el del ejercicio #2) mayor a 40000.

In [7]:
filtered_table=pkm.groupby(["type1","type2"]).filter(lambda row: oakgueda_indicator(row)>=40000)
display(filtered_table)

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
#,,,,,,,,,,,
425,Primal Groudon,Ground,Fire,100,180,160,150,90,90,3,True
799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


In [38]:
print(f"Hay {filtered_table.shape[0]} pokemon(es) que cumplen esta condición, los cuales son:")
temp=filtered_table.loc[:,"name"]
for i in temp.values:
    print(f"{i}")

Hay 2 pokemon(es) que cumplen esta condición, los cuales son:
Primal Groudon
Hoopa Unbound


__Respuesta:__ Hay solo dos pokemones que cumplen esta condición.

## Bonus Track

Honestamente, ¿te causó risa el chiste del profesor Oakgueda? Responde del 1 al 5, donde 1 es equivalente a _"Me dio vergüenza ajena"_ y 5 a _"Me alegró el día y mi existencia en la UTFSM"_.

__Respuesta__: 5
